In [1]:
import os, time, sys
from pathlib import Path

SESSION_NAME = "TESTING"

ROOT = Path().absolute().parent
RAW_INPUT_PATH = Path(f"{ROOT}/data/1_raw_input")
INPUT_PATH = Path(f"{ROOT}/data/2_input")
RESULTS_PATH = Path(f"{ROOT}/data/3_results")

SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")

!mkdir -p {RAW_INPUT_PATH}
!mkdir -p {INPUT_PATH}
!mkdir -p {SESSION_PATH}

print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID TESTING under /mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/3_results


In [ ]:
##### Data Preparation ( raw input -> input folder)
%cd {ROOT}

raw_input_files = [x for x in RAW_INPUT_PATH.glob('**/*') if x.is_file()]
print(f"found {len(raw_input_files)} raw input files")
_f = "/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4" #str(raw_input_files[0])
print(_f)
DOWNSAMPLING_RATE = 2

#!bash scripts/third_party/neuralangelo/run_ffmpeg.sh {{SESSION_NAME}} {{_f}} 2
! ffmpeg -i {_f} -vf "select=not(mod(n\,{DOWNSAMPLING_RATE}))" -vsync vfr -q:v 2 {INPUT_PATH}/%06d.jpg

In [ ]:
# Rename mask files correctly
# POST Processing
extension_name = ".JPEG"
MASK_PATH = f"{INPUT_PATH}/masks"
mask_images = []
for f in os.listdir(MASK_PATH):
    if f.endswith(".png"):
        mask_images.append(f)

in_images = []
for f in os.listdir(INPUT_PATH):
    if f.endswith(extension_name):
        in_images.append(f)

print(mask_images, in_images)

for i, filename in enumerate(mask_images):
    in_file_name = os.path.join(MASK_PATH, filename)
    split_name = os.path.splitext(filename)
    new_file_name = f"{split_name[0]}{extension_name}{split_name[1]}"
    out_file_name = os.path.join(MASK_PATH, f"{new_file_name}")
    print(new_file_name)
    os.rename(in_file_name, out_file_name)

In [2]:
# Sparse Reconstruction (Step 1)
#! bash run_colmap_sparse.sh "{SESSION_PATH}" "{INPUT_PATH}"
! bash run_colmap_sparse_matching_only.sh "{SESSION_PATH}" "{INPUT_PATH}"
#MASK_PATH = f"{INPUT_PATH}/masks"
#! bash run_colmap_sparse_with_masks.sh "{SESSION_PATH}" "{INPUT_PATH}" "{MASK_PATH}"

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700
I0326 19:18:39.835207 109798 misc.cc:198] 
Feature extraction
I0326 19:18:41.503721 109832 feature_extraction.cc:254] Processed file [1/60]
I0326 19:18:41.503768 109832 feature_extraction.cc:257]   Name:            IMG_1140.png
I0326 19:18:41.503798 109832 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0326 19:18:41.503819 109832 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I0326 19:18:41.503825 109832 feature_extraction.cc:289]   Focal Length:    1965.60px
I0326 19:18:41.503836 109832 feature_extraction.cc:302]   Features:        13895
I0326 19:18:42.102051 109832 feature_extraction.cc:254] Processed file [2/60]
I0326 19:18:42.102090 109832 feature_extraction.cc:257]   Name:            IMG_1141.png
I0326 19:18:42.102136 109832 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0326 19:18:42.102147 109832 feature_extraction.cc:286]   Camera:          #2 -

In [ ]:
# Dense Reconstruction IF NEEDED (Step 2)
! bash run_colmap_dense.sh "{SESSION_PATH}"

## Generate Transforms File
Generate this file if you want to have camera, points etc. in a self contained file which you can then use to display everything in a 3D engine like Blender (see my scripts `colmap_pc_importer_ui.py` and `colmap_pc_importer.py` which u can load into Blender)

In [6]:
import sys

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print(sys.path)

generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json_advanced.py"
! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor" --image_dir "{INPUT_PATH}"

#generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json.py"
#! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor"

['/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/scripts', '/home/tworkool/miniconda3/envs/historical-photo-sfm-pipeline/lib/python39.zip', '/home/tworkool/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9', '/home/tworkool/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/lib-dynload', '', '/home/tworkool/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/site-packages', '/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline']
/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline
Fraction of images looking at the center: 0.00.
Fraction of images positioned around the center: 0.77.
Valid fraction of concentric images: 0.00.
Writing data to json file:  /mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/3_results/3_koepenick_rathaus_SfM_agfaisolette/transforms_withpoints.json


In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()

        if exif_data is not None:
            # Decode the EXIF data
            decoded_exif = {TAGS[key]: exif_data[key] for key in exif_data.keys() if key in TAGS and isinstance(exif_data[key], (int, str, bytes))}
            return decoded_exif
        else:
            print("No EXIF data found.")
            return None

    except Exception as e:
        print(f"Error reading EXIF data: {e}")
        return None

# Example usage
image_path = '/mnt/d/dev/python/historical-photo-sfm-pipeline/data/2_input/IMG_0555.jpeg'
exif_data = get_exif_data(image_path)

if exif_data:
    print("EXIF Metadata:")
    for key, value in exif_data.items():
        print(f"{key}: {value}")

In [ ]:
'''
camera = pycolmap.Camera(
    model=camera_model_name_or_id,
    width=width,
    height=height,
    params=params,
)

import pycolmap
reconstruction = pycolmap.Reconstruction("path/to/reconstruction/dir")
print(reconstruction.summary())

for image_id, image in reconstruction.images.items():
    print(image_id, image)

for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

reconstruction.write("path/to/reconstruction/dir/")
'''